In [3]:
import pandas as pd

In [4]:
access = pd.read_csv("/home/jaime/projet_info_3A/private/access.csv",header=0)
id_api = access["Identifiant"][0]
key_api = access["Clé secrète"][0]

In [5]:
import requests 
import os

In [6]:
import emploi_store

In [7]:
client_id = id_api
client_secret = key_api
access_tokens = {}
cached_packages = None
scope = 'api_offresdemploiv2'

In [8]:
# from https://github.com/Cocorico84/employme/blob/855cfcda2c54f42b9c42dcbf4ac87cdf09248b7f/back/manager.py

In [9]:
import requests


url = "https://api.emploi-store.fr/partenaire"

In [10]:
def get_token():
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token'
    r = requests.post(url_token, headers={'Content-Type': 'application/x-www-form-urlencoded'},
                      params={'realm': '/partenaire'},
                      data={'grant_type': 'client_credentials', 'client_id': id_api,
                            'client_secret': key_api,
                            'scope': f'application_{id_api} api_offresdemploiv2 o2dsoffre'})
    data = r.json()
    return data['access_token']


In [35]:
def connection(token,range_str='0-149'):
    url_offers = f'{url}/offresdemploi/v2/offres/search'
    r = requests.get(url=url_offers,params={'range':range_str},   #{'motsCles': 'informatique'} ,# params={'rantge=':range_str}, #,  params={'motsCles': 'informatique'}
        headers={'Authorization': f'Bearer {get_token()}'})
    data = r.json()

    return data

In [16]:
def get_results_as_df(range_str='0-149'):
    
    request_as_dict=connection(get_token(),range_str)

    df = pd.DataFrame.from_dict(request_as_dict['resultats'])

    df = pd.concat([df.drop(['lieuTravail'], axis=1), df['lieuTravail'].apply(pd.Series)], axis=1)
    df = pd.concat([df.drop(['origineOffre'], axis=1), df['origineOffre'].apply(pd.Series)], axis=1)
    df = pd.concat([df.drop(['partenaires'], axis=1), df['partenaires'].apply(pd.Series)], axis=1)
    
    return df #request_as_dict['filtres possibles']


# test une page

In [32]:
df = get_results_as_df('0-3')

{'filtresPossibles': [{'agregation': [{'nbResultats': 1083, 'valeurPossible': 'CCE'}, {'nbResultats': 47059, 'valeurPossible': 'CDD'}, {'nbResultats': 189910, 'valeurPossible': 'CDI'}, {'nbResultats': 61, 'valeurPossible': 'CDS'}, {'nbResultats': 144, 'valeurPossible': 'DDI'}, {'nbResultats': 67, 'valeurPossible': 'DIN'}, {'nbResultats': 3590, 'valeurPossible': 'FRA'}, {'nbResultats': 8275, 'valeurPossible': 'LIB'}, {'nbResultats': 44729, 'valeurPossible': 'MIS'}, {'nbResultats': 1150, 'valeurPossible': 'REP'}, {'nbResultats': 4633, 'valeurPossible': 'SAI'}, {'nbResultats': 50, 'valeurPossible': 'TTI'}], 'filtre': 'typeContrat'}, {'agregation': [{'nbResultats': 39374, 'valeurPossible': '0'}, {'nbResultats': 142701, 'valeurPossible': '1'}, {'nbResultats': 92707, 'valeurPossible': '2'}, {'nbResultats': 25969, 'valeurPossible': '3'}], 'filtre': 'experience'}, {'agregation': [{'nbResultats': 107562, 'valeurPossible': '0'}, {'nbResultats': 33754, 'valeurPossible': '9'}, {'nbResultats': 1594

In [33]:
df

,accessibleTH,alternance,appellationlibelle,competences,contact,dateActualisation,dateCreation,description,dureeTravailLibelle,dureeTravailLibelleConverti,...,typeContrat,typeContratLibelle,codePostal,commune,latitude,libelle,longitude,origine,urlOrigine,0
0,False,False,Ouvrier / Ouvrière en viticulture,"[{'code': '122825', 'exigence': 'S', 'libelle'...",{'coordonnees1': 'Courriel : champagne.maletre...,NaN,2020-04-05T16:06:05+02:00,Nous recherchons une personne pour effectuer l...,35H Horaires normaux,Temps plein,...,CDD,Contrat à durée déterminée - 1 Mois,51500,51112,49.171944,51 - CHAMERY,3.955000,1,https://candidat.pole-emploi.fr/offres/recherc...,NaN
1,False,False,Agent / Agente de maintenance en informatique,"[{'code': '106839', 'exigence': 'S', 'libelle'...",{'coordonnees1': 'Courriel : recrutement@ville...,NaN,2020-04-05T16:04:02+02:00,Doté de bonnes compétences sur l'environnement...,35H Horaires normaux,Temps plein,...,CDD,Contrat à durée déterminée - 6 Mois,30100,30007,44.127222,30 - ALES,4.080833,1,https://candidat.pole-emploi.fr/offres/recherc...,NaN
2,False,False,Massicotier / Massicotière en industrie graphique,"[{'code': '102830', 'exigence': 'S', 'libelle'...",{'coordonnees1': 'Courriel : karine.raguin@mcc...,NaN,2020-04-05T15:55:25+02:00,Assure la découpe des étiquettes dans le respe...,35H Horaires normaux,Temps plein,...,CDI,Contrat à durée indéterminée,44220,44047,47.214722,44 - COUERON,-1.723889,1,https://candidat.pole-emploi.fr/offres/recherc...,NaN
3,NaN,False,Conseiller / Conseillère immobilier,[],NaN,2020-04-05T15:10:04+02:00,2020-04-05T15:10:01+02:00,"Le poste :L'entreprise : SAFTISAFTI, le réseau...",NaN,Non renseigné,...,CDI,Contrat à durée indéterminée,69001,69381,45.768611,69 - LYON 01,4.833611,2,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...


In [31]:
df.to_csv("df_clean_une_page.csv", sep=",", header=False, index=False)

# "tourner" pages

In [60]:
iteration_index = list(range(0,1150,150))

In [61]:
iteration_index

[0, 150, 300, 450, 600, 750, 900, 1050]

In [45]:
iteration_index[-1]

1050

In [48]:
list_dfs = [get_results_as_df(
    str(str(min(i,1000))+'-'+str(min(i+149,1149)))) for i in list(range(0,1150,150))]

In [49]:
all_dfs = pd.concat(list_dfs)

/home/jaime/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [50]:
all_dfs

,accessibleTH,alternance,appellationlibelle,codePostal,commune,competences,contact,dateActualisation,dateCreation,description,...,secteurActiviteLibelle,trancheEffectifEtab,typeContrat,typeContratLibelle,urlOrigine,0,1,deplacementCode,deplacementLibelle,2
0,False,False,Ouvrier / Ouvrière en viticulture,51500,51112,"[{'code': '122825', 'exigence': 'S', 'libelle'...",{'coordonnees1': 'Courriel : champagne.maletre...,NaN,2020-04-05T16:06:05+02:00,Nous recherchons une personne pour effectuer l...,...,Culture de la vigne,3 à 5 salariés,CDD,Contrat à durée déterminée - 1 Mois,https://candidat.pole-emploi.fr/offres/recherc...,NaN,NaN,NaN,NaN,NaN
1,False,False,Agent / Agente de maintenance en informatique,30100,30007,"[{'code': '106839', 'exigence': 'S', 'libelle'...",{'coordonnees1': 'Courriel : recrutement@ville...,NaN,2020-04-05T16:04:02+02:00,Doté de bonnes compétences sur l'environnement...,...,Administration publique générale,1 ou 2 salariés,CDD,Contrat à durée déterminée - 6 Mois,https://candidat.pole-emploi.fr/offres/recherc...,NaN,NaN,NaN,NaN,NaN
2,False,False,Massicotier / Massicotière en industrie graphique,44220,44047,"[{'code': '102830', 'exigence': 'S', 'libelle'...",{'coordonnees1': 'Courriel : karine.raguin@mcc...,NaN,2020-04-05T15:55:25+02:00,Assure la découpe des étiquettes dans le respe...,...,Autre imprimerie (labeur),50 à 99 salariés,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,NaN,NaN,NaN,NaN,NaN
3,NaN,False,Conseiller / Conseillère immobilier,69001,69381,[],NaN,2020-04-05T15:10:04+02:00,2020-04-05T15:10:01+02:00,"Le poste :L'entreprise : SAFTISAFTI, le réseau...",...,NaN,NaN,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,NaN,NaN,NaN,NaN
4,NaN,False,Pharmacien / Pharmacienne,75001,75101,[],NaN,2020-04-05T15:09:53+02:00,2020-04-05T15:09:52+02:00,"L'équipe est à taille humaine, jeune et dynami...",...,NaN,NaN,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,NaN,NaN,NaN,NaN
5,NaN,False,Responsable planification en industrie,45300,45252,[],NaN,2020-04-05T15:09:26+02:00,2020-04-05T15:09:25+02:00,Responsable Planification H/F\nRattaché au Dir...,...,Activités des agences de publicité,NaN,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,{'logo': 'https://www.pole-emploi.fr/static/im...,NaN,NaN,NaN
6,NaN,False,Éducateur / Éducatrice de jeunes enfants,92200,92051,[],NaN,2020-04-05T15:09:23+02:00,2020-04-05T15:09:22+02:00,"En tant qu'éducateur de jeunes enfants H/F, vo...",...,NaN,NaN,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,NaN,NaN,NaN,NaN
7,NaN,True,Technicien / Technicienne réseau informatique,75001,75101,[],NaN,2020-04-05T15:09:22+02:00,2020-04-05T15:09:21+02:00,POSTE : Technicien Systèmes et Réseaux H/F\nPR...,...,NaN,NaN,CDD,Contrat à durée déterminée - 12 Mois,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,NaN,NaN,NaN,NaN
8,NaN,False,Technicien / Technicienne de Service Après-Ven...,13001,13201,[],NaN,2020-04-05T15:09:12+02:00,2020-04-05T15:09:09+02:00,Mon client est à la recherche d'un(e)technicie...,...,NaN,NaN,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,NaN,NaN,NaN,NaN
9,NaN,False,Technicien / Technicienne en prévention des ri...,33000,33063,[],NaN,2020-04-05T15:09:06+02:00,2020-04-05T15:09:06+02:00,En bref : Technicien en prévention des risques...,...,NaN,NaN,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,NaN,NaN,NaN,NaN


In [62]:
all_dfs = all_dfs.astype(str).drop_duplicates()

In [63]:
all_dfs.to_csv("df_max_pages.csv", sep=",", header=False, index=False)

In [58]:
type(all_dfs)

pandas.core.frame.DataFrame

In [59]:
all_dfs.astype(str).drop_duplicates()

,accessibleTH,alternance,appellationlibelle,codePostal,commune,competences,contact,dateActualisation,dateCreation,description,...,secteurActiviteLibelle,trancheEffectifEtab,typeContrat,typeContratLibelle,urlOrigine,0,1,deplacementCode,deplacementLibelle,2
0,False,False,Ouvrier / Ouvrière en viticulture,51500,51112,"[{'code': '122825', 'exigence': 'S', 'libelle'...",{'coordonnees1': 'Courriel : champagne.maletre...,nan,2020-04-05T16:06:05+02:00,Nous recherchons une personne pour effectuer l...,...,Culture de la vigne,3 à 5 salariés,CDD,Contrat à durée déterminée - 1 Mois,https://candidat.pole-emploi.fr/offres/recherc...,nan,nan,nan,nan,nan
1,False,False,Agent / Agente de maintenance en informatique,30100,30007,"[{'code': '106839', 'exigence': 'S', 'libelle'...",{'coordonnees1': 'Courriel : recrutement@ville...,nan,2020-04-05T16:04:02+02:00,Doté de bonnes compétences sur l'environnement...,...,Administration publique générale,1 ou 2 salariés,CDD,Contrat à durée déterminée - 6 Mois,https://candidat.pole-emploi.fr/offres/recherc...,nan,nan,nan,nan,nan
2,False,False,Massicotier / Massicotière en industrie graphique,44220,44047,"[{'code': '102830', 'exigence': 'S', 'libelle'...",{'coordonnees1': 'Courriel : karine.raguin@mcc...,nan,2020-04-05T15:55:25+02:00,Assure la découpe des étiquettes dans le respe...,...,Autre imprimerie (labeur),50 à 99 salariés,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,nan,nan,nan,nan,nan
3,nan,False,Conseiller / Conseillère immobilier,69001,69381,[],nan,2020-04-05T15:10:04+02:00,2020-04-05T15:10:01+02:00,"Le poste :L'entreprise : SAFTISAFTI, le réseau...",...,nan,nan,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,nan,nan,nan,nan
4,nan,False,Pharmacien / Pharmacienne,75001,75101,[],nan,2020-04-05T15:09:53+02:00,2020-04-05T15:09:52+02:00,"L'équipe est à taille humaine, jeune et dynami...",...,nan,nan,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,nan,nan,nan,nan
5,nan,False,Responsable planification en industrie,45300,45252,[],nan,2020-04-05T15:09:26+02:00,2020-04-05T15:09:25+02:00,Responsable Planification H/F\nRattaché au Dir...,...,Activités des agences de publicité,nan,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,{'logo': 'https://www.pole-emploi.fr/static/im...,nan,nan,nan
6,nan,False,Éducateur / Éducatrice de jeunes enfants,92200,92051,[],nan,2020-04-05T15:09:23+02:00,2020-04-05T15:09:22+02:00,"En tant qu'éducateur de jeunes enfants H/F, vo...",...,nan,nan,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,nan,nan,nan,nan
7,nan,True,Technicien / Technicienne réseau informatique,75001,75101,[],nan,2020-04-05T15:09:22+02:00,2020-04-05T15:09:21+02:00,POSTE : Technicien Systèmes et Réseaux H/F\nPR...,...,nan,nan,CDD,Contrat à durée déterminée - 12 Mois,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,nan,nan,nan,nan
8,nan,False,Technicien / Technicienne de Service Après-Ven...,13001,13201,[],nan,2020-04-05T15:09:12+02:00,2020-04-05T15:09:09+02:00,Mon client est à la recherche d'un(e)technicie...,...,nan,nan,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,nan,nan,nan,nan
9,nan,False,Technicien / Technicienne en prévention des ri...,33000,33063,[],nan,2020-04-05T15:09:06+02:00,2020-04-05T15:09:06+02:00,En bref : Technicien en prévention des risques...,...,nan,nan,CDI,Contrat à durée indéterminée,https://candidat.pole-emploi.fr/offres/recherc...,{'logo': 'https://www.pole-emploi.fr/static/im...,nan,nan,nan,nan


In [54]:
all_dfs.groupby(all_dfs.columns.tolist(),as_index=False).size()

TypeError: unhashable type: 'list'

In [52]:
a.duplicated()

TypeError: unhashable type: 'list'